<a href="https://colab.research.google.com/github/Mahmuda-104/Thesis/blob/main/Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
from scipy.io import loadmat

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_path = '/content/drive/My Drive/Data'

In [4]:
import os
import scipy.io
import numpy as np
from scipy import signal

lowcut = 0.5 
highcut = 100  
fs_original = 500
fs_new = 250

control_data = []
control_path = os.path.join(data_path, 'Control')
for foldername in os.listdir(control_path):
    subfolder_path = os.path.join(control_path, foldername)
    control_data.append(subfolder_path)

Control30 = []
for control_data_path in control_data:
    n_epochs = 60 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    control_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{control_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        
        b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        filtered_data = signal.filtfilt(b, a, mat_data1, axis=-1)
        
        # Downsampling
        num_samples_original = filtered_data.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(filtered_data, num_samples_new, axis=-1)
        
        control_arr.append(np.array(downsampled_data))
        
    Control30.append(np.array(control_arr))

Control30 = np.array(Control30)
print( Control30[0].shape)


(60, 60, 1000)


In [5]:
print("Filtered and downsampled data shape:", Control30.shape)

Filtered and downsampled data shape: (30, 60, 60, 1000)


In [6]:
import os
import scipy.io
import numpy as np
from scipy import signal

lowcut = 0.5 
highcut = 100  
fs_original = 500
fs_new = 250  

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed49 = []
for concussed_data_path in concussed_data:
    n_epochs = 60 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    concussed_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        
        b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        filtered_data = signal.filtfilt(b, a, mat_data1, axis=-1)
        
        # Downsampling
        num_samples_original = filtered_data.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(filtered_data, num_samples_new, axis=-1)
        
        concussed_arr.append(np.array(downsampled_data))
        
    Concussed49.append(np.array(control_arr))

Concussed49 = np.array(Concussed49)
print( Concussed49[0].shape)


(60, 60, 1000)


In [7]:
print("Filtered and downsampled data shape:", Concussed49.shape)

Filtered and downsampled data shape: (49, 60, 60, 1000)


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
dataset = np.concatenate((Control30, Concussed49), axis=0)

#(0 for control, 1 for concussed)
labels = np.concatenate((np.zeros(len(Control30)), np.ones(len(Concussed49))))

In [9]:
train_ratio = 0.8  # 80% for training
val_ratio = 0.1    # 10% for validation
test_ratio = 0.1   # 10% for testing

In [10]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

In [11]:
import tensorflow as tf
from tensorflow.keras import layers

# Normalize the input data (if required)
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Reshape the input data based on the expected input shape of the model
input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Build the CNN model
model = tf.keras.Sequential()
model.add(layers.Conv2D(16, kernel_size=(6, 6),strides=2, activation='relu',padding='same', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', strides=1,padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', strides=1,padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))  # num_classes is the number of output classes
from keras.callbacks import EarlyStopping


callback_list = EarlyStopping(monitor = 'val_loss',
                              patience = 20,
                              restore_best_weights=True)

# Compile the model with the desired loss function and optimizer
learning_rate = 0.05  # Specify your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)


Epoch 1/10
2/2 [==============================] - 9s 4s/step - loss: 329.4138 - accuracy: 0.5873 - val_loss: 4.5367 - val_accuracy: 0.3750
Epoch 2/10
2/2 [==============================] - 6s 3s/step - loss: 22.3756 - accuracy: 0.4444 - val_loss: 0.6619 - val_accuracy: 0.6250
Epoch 3/10
2/2 [==============================] - 7s 3s/step - loss: 0.6893 - accuracy: 0.6190 - val_loss: 0.7046 - val_accuracy: 0.3750
Epoch 4/10
2/2 [==============================] - 6s 3s/step - loss: 0.7715 - accuracy: 0.4762 - val_loss: 0.6626 - val_accuracy: 0.6250
Epoch 5/10
2/2 [==============================] - 7s 4s/step - loss: 0.6339 - accuracy: 0.6349 - val_loss: 0.6695 - val_accuracy: 0.6250
Epoch 6/10
2/2 [==============================] - 6s 3s/step - loss: 0.7198 - accuracy: 0.5873 - val_loss: 0.6622 - val_accuracy: 0.6250
Epoch 7/10
2/2 [==============================] - 7s 4s/step - loss: 0.6617 - accuracy: 0.6190 - val_loss: 0.6765 - val_accuracy: 0.6250
Epoch 8/10
2/2 [======================

In [12]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Split the dataset into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

# Normalize the input data (if required)
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Reshape the input data based on the expected input shape of the model
input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Build the CNN model
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with the desired loss function and optimizer
learning_rate = 0.001  # Specify your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)


Epoch 1/10
2/2 [==============================] - 10s 5s/step - loss: 0.6040 - accuracy: 0.6508 - val_loss: 0.2960 - val_accuracy: 1.0000
Epoch 2/10
2/2 [==============================] - 7s 4s/step - loss: 0.2325 - accuracy: 0.9683 - val_loss: 0.0773 - val_accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - 8s 4s/step - loss: 0.0847 - accuracy: 0.9841 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 8s 4s/step - loss: 0.1053 - accuracy: 0.9841 - val_loss: 0.0048 - val_accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 8s 3s/step - loss: 0.0797 - accuracy: 0.9841 - val_loss: 0.4371 - val_accuracy: 0.3750
Epoch 6/10
2/2 [==============================] - 8s 4s/step - loss: 0.4096 - accuracy: 0.6032 - val_loss: 2.1695e-04 - val_accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 7s 4s/step - loss: 0.2277 - accuracy: 0.9841 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 8/10
2/2 [====================

In [13]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Split the dataset into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

# Normalize the input data (if required)
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Reshape the input data based on the expected input shape of the model
input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Build the CNN model
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

# Compile the model with the desired loss function and optimizer
learning_rate = 0.001  # Specify your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)


Epoch 1/10
2/2 [==============================] - 9s 4s/step - loss: 37.7335 - accuracy: 0.4921 - val_loss: 0.1992 - val_accuracy: 0.8750
Epoch 2/10
2/2 [==============================] - 9s 4s/step - loss: 3.4921 - accuracy: 0.7302 - val_loss: 0.7001 - val_accuracy: 0.3750
Epoch 3/10
2/2 [==============================] - 8s 5s/step - loss: 0.7685 - accuracy: 0.3810 - val_loss: 0.6151 - val_accuracy: 0.3750
Epoch 4/10
2/2 [==============================] - 7s 4s/step - loss: 0.6603 - accuracy: 0.3810 - val_loss: 0.5602 - val_accuracy: 0.3750
Epoch 5/10
2/2 [==============================] - 8s 4s/step - loss: 0.6128 - accuracy: 0.3651 - val_loss: 0.5185 - val_accuracy: 0.3750
Epoch 6/10
2/2 [==============================] - 7s 4s/step - loss: 0.5269 - accuracy: 0.3651 - val_loss: 0.4902 - val_accuracy: 0.3750
Epoch 7/10
2/2 [==============================] - 8s 4s/step - loss: 0.4910 - accuracy: 0.3810 - val_loss: 0.4565 - val_accuracy: 0.3750
Epoch 8/10
2/2 [========================